In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [182]:
df_location = pd.read_csv("scrapper/UK_raw_data.csv")
df_location.drop_duplicates(inplace=True)
df_location.columns
df_location['Location'].unique()

array(['London, England', 'Bradford, Yorkshire and the Humber, England',
       'Edinburgh, Scotland', 'Leeds, England',
       'Belfast, Northern Ireland', 'Poole, England',
       'Sunderland, North East England, England', 'Barnwood, England',
       'Remote', 'Manchester, England',
       'Cambridge, East of England, England', 'High Wycombe, England',
       'Maidenhead, England', 'Nottingham, England', 'Melbourn, England',
       'England', 'Newport, Newport, Wales', 'Coventry, England',
       'Richmond upon Thames, England', 'Norwich, England',
       'Sedgefield, England', 'Slimbridge, England',
       'Crawley, West Sussex, South East England, England',
       'Loughborough, England', 'Kingston upon Hull, England',
       'United Kingdom', 'Reading, England',
       'Craigavon, Northern Ireland', 'Oxford, England',
       'Bristol, England', 'Guildford, England', 'Isleworth, England',
       'Abingdon, England', 'Berkhamsted, England',
       'Newcastle upon Tyne, England', 'Ux

In [194]:
df_location = df_location[df_location['Salary Estimate'] != '-1'] 
df_location.reset_index(drop=True, inplace=True)

In [195]:
df_location.shape

(880, 17)

In [196]:
def sparse_country(x):
    country=''
    x = x.split(',')
    if len(x)==3:
        country = x[2]
    elif len(x) == 2:
        country = x[1]
    return country

def sparse_region(x):
    region=''
    x=x.split(',')
    if len(x)==3:
        region = x[1]
    elif len(x)==2:
        region = 'Unknown'
    return region

In [197]:
df_location['cities'] = df_location['Location'].apply(lambda x:x.split(',')[0])
df_location['region'] = df_location['Location'].apply(sparse_region)
df_location['country'] = df_location['Location'].apply(sparse_country)
print(df_location['country'])
print(df_location['region'])
print(df_location['cities'])

0       England
1       England
2       England
3       England
4       England
         ...   
875     England
876            
877     England
878       Wales
879     England
Name: country, Length: 880, dtype: object
0                        Unknown
1       Yorkshire and the Humber
2                        Unknown
3                        Unknown
4                        Unknown
                 ...            
875                      Unknown
876                             
877                      Unknown
878                      Gwynedd
879                      Unknown
Name: region, Length: 880, dtype: object
0                  London
1                Bradford
2                  London
3                  London
4                  London
              ...        
875                London
876    Greater Manchester
877             Bracknell
878                Bangor
879             Guildford
Name: cities, Length: 880, dtype: object


In [198]:
df = pd.read_csv("cleaned_data.csv")
df.drop('Unnamed: 0', axis=1, inplace=True)
df['City'] = df_location['cities']
df['Region'] = df_location['region']
df['Country'] = df_location['country']
print(df.info(),'\n===============================================')
print(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 880 entries, 0 to 879
Data columns (total 24 columns):
Company Name         880 non-null object
job_title            880 non-null object
seniority            880 non-null object
min_salary           880 non-null int64
max_salary           880 non-null int64
Rating               880 non-null float64
python               880 non-null int64
sql                  880 non-null int64
r_studio             880 non-null int64
spark                880 non-null int64
aws                  880 non-null int64
excel                880 non-null int64
sector               880 non-null object
age                  880 non-null int64
Revenue              880 non-null object
Type of ownership    880 non-null object
description_len      880 non-null int64
min_employees        880 non-null int64
max_employees        880 non-null int64
avg_size             880 non-null float64
avg_salary           880 non-null float64
City                 880 non-null object
Re

In [199]:
df_model = df[['avg_salary','job_title','City','Region', 'Country', 'seniority','Rating', 'python', 'sql', 'r_studio', 
               'spark', 'aws', 'excel','sector', 'age', 'Revenue', 'Type of ownership', 
               'description_len', 'avg_size']]

In [200]:
df_dummy = pd.get_dummies(df_model)

In [201]:
df_dummy.shape

(880, 205)

In [202]:
# train test split
from sklearn.model_selection import train_test_split
X = df_dummy.drop('avg_salary', axis=1)
y = df_dummy.avg_salary.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# multiple linear regression
import statsmodels.api as sm
X_sm = sm.add_constant(X)
stat_model = sm.OLS(y,X_sm)
print(stat_model.fit().summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.257
Model:                            OLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     1.356
Date:                Sat, 02 Jan 2021   Prob (F-statistic):            0.00383
Time:                        02:22:20   Log-Likelihood:                -3694.3
No. Observations:                 880   AIC:                             7749.
Df Residuals:                     700   BIC:                             8609.
Df Model:                         179                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

C:\Users\Saurabh\anaconda3\envs\webscrape\lib\site-packages\numpy\core\fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [ ]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import cross_val_score